In [1]:
import sys
import pandas as pd
import numpy as np
pd.options.display.max_columns = None
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_absolute_error
import time

sys.path.append('../')

### Binary

In [2]:
train = pd.read_parquet('/www/dslib/spark_sota_modeling/dataset/home-credit-default-risk/train.parquet')
target_col = 'target'
index_col = 'sk_id_curr'
train, test = train_test_split(train, test_size=0.2, random_state=42, stratify=train[target_col])
X_train = train.drop(columns=[target_col, index_col])
y_train = train[target_col]
X_test = test.drop(columns=[target_col, index_col])
y_test = test[target_col]
with open('/www/dslib/spark_sota_modeling/dataset/home-credit-default-risk/categorical_features.txt', 'r') as f:
    categorical_features = [line.strip() for line in f.readlines()]
len(categorical_features)

16

In [ ]:
from models.estimators.tabnet_estimator import TabNetBinary

model = TabNetBinary(
    d_model = 8,
    n_steps = 5,
    decision_dim = 512,
    n_shared = 4,
    n_independent = 4,
    glu_dropout = 0.2,
    dropout_emb = 0.6,
    glu_norm = 'batch',
    gamma = 1.5,
    lambda_sparse = 1e-4,
    batch_size = 1024,
    epochs = 200,
    learning_rate = 0.01,
    early_stopping_patience = 10,
    weight_decay = 1e-5,
    reducelronplateau_patience = 5,
    reducelronplateau_factor = 0.7,
    verbose = True,
    random_state = 42,
)
model.fit(X_train, y_train, eval_set=(X_test, y_test), eval_metric='roc_auc', mode='max', cat_features=categorical_features)
y_pred_proba = model.predict_proba(X_test, cat_features=categorical_features)
roc_auc_score(y_test, y_pred_proba[:,1])

In [ ]:
from models.estimators.tabnet_estimator import TabNetBinary

for n_shared in [1, 2, 3, 4, 5, 6, 7, 8, 9]:
    start_time = time.time()
    model = TabNetBinary(
        d_model = 8,
        n_steps = 5,
        decision_dim = 64,
        n_shared = n_shared,
        n_independent = n_shared,
        glu_dropout = 0.3,
        dropout_emb = 0.3,
        glu_norm = 'batch',
        gamma = 1.5,
        lambda_sparse = 1e-5,
        batch_size = 1024,
        epochs = 200,
        learning_rate = 0.01,
        early_stopping_patience = 10,
        weight_decay = 1e-5,
        reducelronplateau_patience = 5,
        reducelronplateau_factor = 0.7,
        verbose = False,
        random_state = 42,
    )

    model.fit(X_train, y_train, eval_set=(X_test, y_test), eval_metric='roc_auc', mode='max', 
              cat_features=categorical_features)
    y_pred_proba = model.predict_proba(X_test, cat_features=categorical_features)
    formatted = time.strftime("%H:%M:%S", time.gmtime(time.time()-start_time))
    print(
        f"n_shared: {n_shared}, "
        f"ROC AUC: {roc_auc_score(y_test, y_pred_proba[:,1]):.4f}, "
        f"Time: {formatted}"
    )

### Multiclass

In [2]:
train = pd.read_parquet('/www/dslib/spark_sota_modeling/dataset/forest-cover-type/train.parquet')
target_col = 'cover_type'
train, test = train_test_split(train, test_size=0.2, random_state=42, stratify=train[target_col])
X_train = train.drop(columns=[target_col])
y_train = train[target_col]
X_test = test.drop(columns=[target_col])
y_test = test[target_col]
with open('/www/dslib/spark_sota_modeling/dataset/forest-cover-type/categorical_features.txt', 'r') as f:
    categorical_features = [line.strip() for line in f.readlines()]
len(categorical_features)

44

In [ ]:
n_classes=train[target_col].nunique(),
d_model = 8,
n_steps = 6,
decision_dim = 128,
n_shared = 2,
n_independent = 2,
glu_dropout = 0.1,
dropout_emb = 0.1,
glu_norm = 'batch',
gamma = 1.5,
lambda_sparse = 1e-3,
batch_size = 1024,
epochs = 200,
learning_rate = 0.025,
early_stopping_patience = 15,
weight_decay = 1e-5,
reducelronplateau_patience = 5,
reducelronplateau_factor = 0.5,
verbose = True,
random_state = 42,
momentum=0.6
# 0.9256215416125229

n_classes=train[target_col].nunique(),
glu_dropout = 0.05,
dropout_emb = 0.05,
lambda_sparse = 1e-3,
momentum=0.1,
# 0.942256224021755

lambda_sparse = 1e-4,
momentum=0.1,
# 0.9434954347133895

lambda_sparse = 1e-5,
momentum=0.1,
# 0.9424799703966334

lambda_sparse = 1e-4,
learning_rate = 0.01,
momentum=0.1,
# 0.9477035876870649

momentum=0.6,
batch_size = 1024,
# 0.9477035876870649

momentum=0.1,
batch_size = 2048, # 16384
virtual_batch_size=512, # 512 m=0.01
# 0.946765574038536

momentum=0.1,
batch_size = 2048, # 16384
virtual_batch_size=512, # 512 m=0.1
# 0.9467053346299149

momentum=0.7,
batch_size = 2048, # 16384
virtual_batch_size=512, # 512 m=0.1

In [ ]:
from models.estimators.tabnet_estimator import TabNetMulticlass

model = TabNetMulticlass(
    n_classes=train[target_col].nunique(),
    d_model = 8,
    n_steps = 6, # 5
    decision_dim = 128, # 64 + 64
    n_shared = 2, # 2
    n_independent = 2, # 2
    glu_dropout = 0.05,
    dropout_emb = 0.05,
    gamma = 1.5, # 1.5
    lambda_sparse = 0.0001, # 0.0001
    batch_size = 2048, # 16384
    virtual_batch_size=512, # 512
    momentum = 0.7, # 0.7
    epochs = 200, 
    learning_rate = 0.01, # 0.02
    early_stopping_patience = 15,
    weight_decay = 1e-5,
    reducelronplateau_patience = 5,
    reducelronplateau_factor = 0.5,
    verbose = True,
    random_state = 42,
)

model.fit(X_train, y_train, eval_set=(X_test, y_test), cat_features=categorical_features)

y_pred = model.predict(X_test, cat_features=categorical_features)
y_pred_proba = model.predict_proba(X_test, cat_features=categorical_features)
accuracy_score(y_test, y_pred)

Начинаем обучение на cuda...


Training:   0%|          | 0/227 [00:00<?, ?it/s]

Validation:   0%|          | 0/57 [00:00<?, ?it/s]

Epoch 1/200, Train loss: 0.5643, Train accuracy: 0.7601, Val loss: 0.4188, Val accuracy: 0.8213
---> Сохранена лучшая модель (Эпоха 1) с Val accuracy: 0.8213


Training:   0%|          | 0/227 [00:00<?, ?it/s]

...

Epoch 73/200, Train loss: 0.1878, Train accuracy: 0.9219, Val loss: 0.1351, Val accuracy: 0.9463
Нет улучшения 2/15 эпох.


Training:   0%|          | 0/227 [00:00<?, ?it/s]

Validation:   0%|          | 0/57 [00:00<?, ?it/s]

Epoch 74/200, Train loss: 0.1859, Train accuracy: 0.9228, Val loss: 0.1352, Val accuracy: 0.9464
Нет улучшения 3/15 эпох.


Training:   0%|          | 0/227 [00:00<?, ?it/s]

### Regression

In [2]:
train = pd.read_parquet('/www/dslib/spark_sota_modeling/dataset/allstate-claims-severity/train.parquet')
target_col = 'loss'
index_col = 'id'
train, test = train_test_split(train, test_size=0.2, random_state=42)
X_train = train.drop(columns=[target_col, index_col])
y_train = train[target_col]
X_test = test.drop(columns=[target_col, index_col])
y_test = test[target_col]
with open('/www/dslib/spark_sota_modeling/dataset/forest-cover-type/categorical_features.txt', 'r') as f:
    categorical_features = [line.strip() for line in f.readlines()]
len(categorical_features)

44

In [4]:
from models.estimators.tabnet_estimator import TabNetRegressor
model = TabNetRegressor(
    verbose=True,
    dropout=0.4,
    early_stopping_patience=10,
    batch_size=8192,
    virtual_batch_size=128,
    learning_rate=0.005,
    epochs=1000,
    dynamic_emb_size=True,
    n_glu_layers=4,
    n_steps=8,
)

model.fit(X_train, y_train, eval_set=(X_test, y_test), eval_metric='mae', mode='min', cat_features=categorical_features)
y_pred = model.predict(X_test, cat_features=categorical_features)
mean_absolute_error(y_test, y_pred)

Начинаем обучение на cuda...


Training:   0%|          | 0/19 [00:00<?, ?it/s]

Validation:   0%|          | 0/5 [00:00<?, ?it/s]

Epoch 1/1000, Train loss: 17736628.4211, Train mae: 3037.8191, Val loss: 17156994.6000, Val mae: 3020.7126
Сохраняем лучшую модель с метрикой mae: 3020.7126


Training:   0%|          | 0/19 [00:00<?, ?it/s]

...

Training:   0%|          | 0/19 [00:00<?, ?it/s]

Validation:   0%|          | 0/5 [00:00<?, ?it/s]

Epoch 47/1000, Train loss: 2834983.6711, Train mae: 1133.0852, Val loss: 3755053.0000, Val mae: 1187.2515
Останавливаем обучение из-за отсутствия улучшений в течение 10 эпох
Загружена лучшая модель


Predicting:   0%|          | 0/5 [00:00<?, ?it/s]

1175.2066374185233

In [ ]:
1185.3089724793513
1182.7640692897073
1175.2066374185233